In [438]:
import bs4
import requests
import html5lib
import time
import json

In [409]:
base_url = "http://www.ilga.gov"

In [410]:
leg_url = base_url + "/legislation/"

In [411]:
print(leg_url)

http://www.ilga.gov/legislation/


In [412]:
r = requests.get(leg_url)

In [413]:
html_text = r.text.encode('iso-8859-1')

In [414]:
soup = bs4.BeautifulSoup(html_text, "html5lib")

In [415]:
links = soup.find_all("a")

In [416]:
mid_links = []
for a in links:
    if a.has_attr("href") and a['href'].startswith('grplist.asp'):
        mid_url = leg_url + a['href']
        mid_r = requests.get(mid_url)
        mid_html_text = mid_r.text.encode('iso-8859-1')
        mid_soup = bs4.BeautifulSoup(mid_html_text, "html5lib")
        mid_links.extend(mid_soup.find_all("a"))
        

In [417]:
sbill_links = []
hbill_links = []
for mid_a in mid_links:
    if mid_a.text.startswith("SB"):
        bill_url = base_url + mid_a['href']
        sbill_links.append(bill_url)
    elif mid_a.text.startswith("HB"):
        bill_url = base_url + mid_a['href']
        hbill_links.append(bill_url)

In [418]:
len(sbill_links)

3898

In [419]:
len(hbill_links)

5669

In [420]:
r_test = requests.get(sbill_links[0])
html_test = r_test.text.encode('iso-8859-1')
soup_test = bs4.BeautifulSoup(html_test, "html5lib")

In [421]:
soup_test

<html lang="en"><!-- Trigger/Open The Modal --><head></head><body alink="#9933FF" bgcolor="#FFFFFF" leftmargin="0" link="#3366FF" onload="document.NumString.NumSearch.focus();" text="#000000" topmargin="0" vlink="#663366"><div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
<button id="myBtn" style="color: white; background-color: navy; display: block">Translate Website</button></div>
<!-- The Modal -->
<div class="modal" id="myModal" style="display: none">
  <!-- Modal content -->
  <div class="modal-content">
      <div class="modal-header"><h3>
    <span class="close">×</span></h3></div>    
    <p>The Illinois General Assembly offers the Google Translate™ service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
    <p>Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
    <p>The English l

In [422]:
bill_info = {}
soup_test.find_all("span")

[<span class="close">×</span>,
 <span style="white-space: nowrap"><a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a></span>,
 <span class="heading">Bill Status of SB0001</span>,
 <span class="content"><i>101st General Assembly</i></span>,
 <span class="heading2">Short Description:</span>,
 <span class="content">MINIMUM WAGE/INCOME TAX CREDIT</span>,
 <span class="heading2">Senate Sponsors</span>,
 <span class="content">Sen. </span>,
 <span class="content"> and </span>,
 <span class="heading2">House Sponsors</span>,
 <span class="content">(Rep. </span>,
 <span class="content"> and </span>,
 <span class="heading2">Last Action</span>,
 <span class="heading2">Statutes Amended In Order of Appearance</span>,
 <span class="heading2">Synopsis As Introduced</span>,
 <span class="content n

Bill Number

In [423]:
bill_number = soup_test.title.text.split()[-1]
bill_info[bill_number] = {}
print(bill_info)

{'SB0001': {}}


Bill Title (Short Description)

In [424]:
heading_tags = soup_test.find_all('span', class_ = "heading2")
for tag in heading_tags:
    if tag.text.startswith("Short Description"):
        description_tag = tag
        break

In [425]:
bill_short_descr = description_tag.next.next.next.text
bill_info[bill_number]['short description'] = bill_short_descr
print(bill_info)

{'SB0001': {'short description': 'MINIMUM WAGE/INCOME TAX CREDIT'}}


Senate Sponsors

In [426]:
next_tag = None
for tag in heading_tags:
    if tag.text.startswith("Senate Sponsors"):
        next_tag = tag.next_sibling.next_sibling
        break
    

In [427]:
senate_sponsors = []
while True:
    if next_tag is None: break
    if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
        senate_sponsors.append(next_tag.text)
    next_tag = next_tag.next_sibling
    if next_tag in heading_tags: break

In [428]:
bill_info[bill_number]['senate sponsors'] = senate_sponsors
print(bill_info)

{'SB0001': {'short description': 'MINIMUM WAGE/INCOME TAX CREDIT', 'senate sponsors': ['Kimberly A. Lightford', 'Jacqueline Y. Collins', 'Antonio Muñoz', 'Iris Y. Martinez', 'Mattie Hunter', 'Patricia Van Pelt', 'Robert Peters', 'Ann Gillespie', 'Ram Villivalam', 'Omar Aquino', 'Martin A. Sandoval', 'Terry Link', 'Elgie R. Sims, Jr.', 'Toi W. Hutchinson', 'Cristina Castro', 'Emil Jones, III', 'Christopher Belt']}}


House Sponsors

In [429]:
next_tag = None
for tag in heading_tags:
    if tag.text.startswith("House Sponsors"):
        next_tag = tag.next_sibling.next_sibling
        break 

In [430]:
house_sponsors = []
while True:
    if next_tag is None: break
    if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
        house_sponsors.append(next_tag.text)
    next_tag = next_tag.next_sibling
    if next_tag in heading_tags: break

In [431]:
bill_info[bill_number]['house sponsors'] = house_sponsors
print(bill_info)

{'SB0001': {'short description': 'MINIMUM WAGE/INCOME TAX CREDIT', 'senate sponsors': ['Kimberly A. Lightford', 'Jacqueline Y. Collins', 'Antonio Muñoz', 'Iris Y. Martinez', 'Mattie Hunter', 'Patricia Van Pelt', 'Robert Peters', 'Ann Gillespie', 'Ram Villivalam', 'Omar Aquino', 'Martin A. Sandoval', 'Terry Link', 'Elgie R. Sims, Jr.', 'Toi W. Hutchinson', 'Cristina Castro', 'Emil Jones, III', 'Christopher Belt'], 'house sponsors': ['Will Guzzardi', 'Marcus C. Evans, Jr.', 'Jay Hoffman', 'Delia C. Ramirez', 'Emanuel Chris Welch', 'Jaime M. Andrade, Jr.', 'Aaron M. Ortiz', 'Justin Slaughter', 'Celina Villanueva', 'Sara Feigenholtz', 'Yehiel M. Kalish', 'Thaddeus Jones', 'Anne Stava-Murray', 'Mary E. Flowers', 'Luis Arroyo']}}


Synopsis

In [432]:
synopsis_list = []
for tag in soup_test.find_all('span', class_ = "content notranslate"):
    synopsis_list.append(tag.text.strip())
synopsis = "  ".join(synopsis_list)  

In [433]:
bill_info[bill_number]['synopsis'] = synopsis.strip()
print(bill_info)

{'SB0001': {'short description': 'MINIMUM WAGE/INCOME TAX CREDIT', 'senate sponsors': ['Kimberly A. Lightford', 'Jacqueline Y. Collins', 'Antonio Muñoz', 'Iris Y. Martinez', 'Mattie Hunter', 'Patricia Van Pelt', 'Robert Peters', 'Ann Gillespie', 'Ram Villivalam', 'Omar Aquino', 'Martin A. Sandoval', 'Terry Link', 'Elgie R. Sims, Jr.', 'Toi W. Hutchinson', 'Cristina Castro', 'Emil Jones, III', 'Christopher Belt'], 'house sponsors': ['Will Guzzardi', 'Marcus C. Evans, Jr.', 'Jay Hoffman', 'Delia C. Ramirez', 'Emanuel Chris Welch', 'Jaime M. Andrade, Jr.', 'Aaron M. Ortiz', 'Justin Slaughter', 'Celina Villanueva', 'Sara Feigenholtz', 'Yehiel M. Kalish', 'Thaddeus Jones', 'Anne Stava-Murray', 'Mary E. Flowers', 'Luis Arroyo'], 'synopsis': 'Amends the Minimum Wage Law. Makes a technical change in a Section concerning the short title.  Replaces everything after the enacting clause. Amends the Illinois Income Tax Act and the Minimum Wage Law. Provides for an increase in the minimum wage and f

In [434]:
t0 = time.time()

In [435]:
for link in sbill_links[1:100]:
    r_bill = requests.get(link)
    html_bill = r_bill.text.encode('iso-8859-1')
    soup_bill = bs4.BeautifulSoup(html_bill, "html5lib")
    
    #Bill number
    bill_number = soup_bill.title.text.split()[-1]
    bill_info[bill_number] = {}
    
    #Bill description
    heading_tags = soup_bill.find_all('span', class_ = "heading2")
    for tag in heading_tags:
        if tag.text.startswith("Short Description"):
            description_tag = tag
            break
    bill_short_descr = description_tag.next.next.next.text
    bill_info[bill_number]['short description'] = bill_short_descr
    
    #Senate sponsors
    next_tag = None
    for tag in heading_tags:
        if tag.text.startswith("Senate Sponsors"):
            next_tag = tag.next_sibling.next_sibling
            break
    senate_sponsors = []
    while True:
        if next_tag is None: break
        if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
            senate_sponsors.append(next_tag.text)
        next_tag = next_tag.next_sibling
        if next_tag in heading_tags: break
    bill_info[bill_number]['senate sponsors'] = senate_sponsors
    
    #House sponsors
    next_tag = None
    for tag in heading_tags:
        if tag.text.startswith("House Sponsors"):
            next_tag = tag.next_sibling.next_sibling
            break
    house_sponsors = []
    while True:
        if next_tag is None: break
        if isinstance(next_tag, bs4.element.Tag) and next_tag.has_attr("href"):
            house_sponsors.append(next_tag.text)
        next_tag = next_tag.next_sibling
        if next_tag in heading_tags: break
    bill_info[bill_number]['house sponsors'] = house_sponsors
    
    #Synopsis
    synopsis_list = []
    for tag in soup_bill.find_all('span', class_ = "content notranslate"):
        synopsis_list.append(tag.text.strip())
    synopsis = "  ".join(synopsis_list)
    bill_info[bill_number]['synopsis'] = synopsis.strip()
delta_t = time.time() - t0    

Export the bill info to a json file

In [439]:
with open('bill_info.json', 'w') as f:
    json.dump(bill_info, f)